# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [12]:
#new
context_user = context.copy()
print(return_CCRMSQL("Who is the highest-paid employee in the company?", context_user))

```sql
SELECT employee_name, salary
FROM employees
ORDER BY salary DESC
LIMIT 1;
```


In [13]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Who is the highest-paid employee in the company?", old_context_user))

This is your SQL:
```sql
SELECT e.name, MAX(s.salary) AS highest_earnings
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr;
```

This SQL query selects the employee's name and their highest salary by joining the "employees" table with the "salary" table on the employee ID. The MAX() function is used to find the highest earnings.


In [14]:
#new
print(return_CCRMSQL("Which employee has the longest tenure based on their educational background?", context_user))

```sql
SELECT employee_name, MAX(tenure) AS longest_tenure
FROM employees
GROUP BY employee_name
ORDER BY longest_tenure DESC
LIMIT 1;
```


In [15]:
#old
print(return_CCRMSQL("Which employee has the longest tenure based on their educational background?", old_context_user))

This is your SQL:
```sql
SELECT e.name, MAX(s.Years) AS longest_tenure
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
GROUP BY e.name
ORDER BY longest_tenure DESC
LIMIT 1;
```

This SQL query retrieves the employee with the longest tenure based on their educational background. It joins the "employees" and "studies" tables on the ID_usr column, calculates the maximum number of years of education for each employee, orders the results in descending order of tenure, and limits the output to the employee with the longest tenure.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [16]:
# Question 1
context_user = context.copy()
print(return_CCRMSQL("Who is the highest-paid employee in the company?", context_user))

# Question 2
context_user = context.copy()
print(return_CCRMSQL("Which employee has the wildest office fashion sense?", context_user))

# Question 3
context_user = context.copy()
print(return_CCRMSQL("Show me the employee who spends the most time perfecting their latte art.", context_user))

# Question 4
context_user = context.copy()
print(return_CCRMSQL("Identify the employee with the most unconventional desk setup.", context_user))

# Question 5
context_user = context.copy()
print(return_CCRMSQL("Who among the employees is most likely to win a Nobel Prize?", context_user))

# Question 6
context_user = context.copy()
print(return_CCRMSQL("Which employee takes the longest lunch breaks?", context_user))

# Question 7
context_user = context.copy()
print(return_CCRMSQL("Find the employee who has attended the most company-sponsored karaoke nights.", context_user))

# Question 8
context_user = context.copy()
print(return_CCRMSQL("Who is the employee with the most extensive collection of office plants?", context_user))

# Question 9
context_user = context.copy()
print(return_CCRMSQL("Identify the employee who is most likely to organize an impromptu dance-off in the office.", context_user))

# Question 10
context_user = context.copy()
print(return_CCRMSQL("Show me the employee who has the most creative use of sticky notes on their monitor.", context_user))


```sql
SELECT employee_name, salary
FROM employees
ORDER BY salary DESC
LIMIT 1;
```
To determine which employee has the wildest office fashion sense, we can query the employee table and order the results by the "fashion_sense" column in descending order. The employee with the highest value in the "fashion_sense" column can be considered to have the wildest office fashion sense.

```sql
SELECT employee_name
FROM employee
ORDER BY fashion_sense DESC
LIMIT 1;
```
```sql
SELECT employee_name
FROM latte_art
ORDER BY time_spent DESC
LIMIT 1;
```
To identify the employee with the most unconventional desk setup, we would need a table that stores information about the desk setup of each employee. Let's create a table named `desk_setup` for this purpose:

```sql
CREATE TABLE desk_setup (
    employee_id INT PRIMARY KEY,
    setup_description TEXT
);
```

We can then query this table to find the employee with the most unconventional desk setup.
To determine which employee is most likely to win a

In [17]:
# Question 1
old_context_user = old_context.copy()
print(return_CCRMSQL("Who is the highest-paid employee in the company?", old_context_user))

# Question 2
old_context_user = old_context.copy()
print(return_CCRMSQL("Which employee has the fanciest job title?", old_context_user))

# Question 3
old_context_user = old_context.copy()
print(return_CCRMSQL("Can you find the employee who spends the most on coffee?", old_context_user))

# Question 4
old_context_user = old_context.copy()
print(return_CCRMSQL("Who has the most unique office desk decor?", old_context_user))

# Question 5
old_context_user = old_context.copy()
print(return_CCRMSQL("Show me the employee with the most vacation days accumulated.", old_context_user))

# Question 6
old_context_user = old_context.copy()
print(return_CCRMSQL("Identify the employee most likely to win the next office Olympics.", old_context_user))

# Question 7
old_context_user = old_context.copy()
print(return_CCRMSQL("Who has the longest commute to work among the employees?", old_context_user))

# Question 8
old_context_user = old_context.copy()
print(return_CCRMSQL("Which employee has the highest number of post-it notes on their desk?", old_context_user))

# Question 9
old_context_user = old_context.copy()
print(return_CCRMSQL("Find the employee with the most unread emails in their inbox.", old_context_user))

# Question 10
old_context_user = old_context.copy()
print(return_CCRMSQL("Who is the employee most likely to have a pet iguana at home?", old_context_user))


This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name and salary of the highest-paid employee in the company by joining the "employees" and "salary" tables on the employee ID, then ordering the results by salary in descending order and limiting the output to the first row.
This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query selects the employee with the highest salary by joining the "employees" and "salary" tables on the employee ID. It then orders the result by salary in descending order and limits the output to only the top result, which represents the employee with the fanciest job title based on salary.
This is your SQL:
```sql
SELECT e.name, SUM(p.amount) AS total_spent
FROM employees e
JOIN purchases p ON e.ID_usr = p.ID_usr
WHERE p.item = 'coffee'
GROUP BY e.n